Install Required Libraries

In [ ]:
!pip install ucimlrepo pandas numpy matplotlib seaborn scikit-learn tensorflow xgboost

Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from ucimlrepo import fetch_ucirepo

Fetch Dataset

In [ ]:
# Fetch dataset
metro_interstate_traffic_volume = fetch_ucirepo(id=492)

# Features and target
X = metro_interstate_traffic_volume.data.features
y = metro_interstate_traffic_volume.data.targets

# Combine features and target for EDA
data = pd.concat([X, y], axis=1)

Display Dataset Information

In [ ]:
print(metro_interstate_traffic_volume.metadata)
print(metro_interstate_traffic_volume.variables)

Check for Missing Values

In [ ]:
print(data.isnull().sum())

In [ ]:

data['holiday'] = data['holiday'].fillna('None')

Handle Missing Values

In [ ]:
# Handle Missing Values for numeric columns only
for column in data.select_dtypes(include=np.number).columns:
    data.loc[:, column] = data[column].fillna(data[column].median())

In [ ]:
print(data.isnull().sum())

Exploratory Data Analysis (EDA) - Summary Statistics

In [ ]:
print(data.describe())

EDA - Traffic Volume Distribution

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data['traffic_volume'], bins=50, kde=True)
plt.title('Traffic Volume Distribution')
plt.xlabel('Traffic Volume')
plt.ylabel('Frequency')
plt.show()

EDA - Correlation Heatmap

In [ ]:
plt.figure(figsize=(12, 8))
# Calculate correlation only for numeric columns
numeric_data = data.select_dtypes(include=np.number)
sns.heatmap(numeric_data.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

As dataset seems too much imbalanced and missing values we will now clean it

Feature Engineering - Extract Time-Based Features

In [ ]:
data['date_time'] = pd.to_datetime(data['date_time'])
data['hour'] = data['date_time'].dt.hour
data['day_of_week'] = data['date_time'].dt.dayofweek
data['month'] = data['date_time'].dt.month

Feature Engineering - Encode Categorical Variables

In [ ]:
data = pd.get_dummies(data, columns=['weather_main', 'holiday'], drop_first=True)

Drop Unnecessary Columns

In [ ]:
data.drop(['date_time'], axis=1, inplace=True)

 Split Data into Training and Testing Sets

In [ ]:
X = data.drop('traffic_volume', axis=1)
y = data['traffic_volume']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Normalize Numerical Features

In [ ]:
metro_interstate_traffic_volume = fetch_ucirepo(id=492)
X = metro_interstate_traffic_volume.data.features
# The target variable 'y' should be assigned to the 'targets' attribute.
y = metro_interstate_traffic_volume.data.targets
data = pd.concat([X, y], axis=1)
encoded_data = pd.get_dummies(data, columns=['weather_main', 'holiday', 'weather_description'], drop_first=True)
encoded_data.drop(['date_time'], axis=1, inplace=True)
#  Split Data into Training and Testing Sets
X = encoded_data.drop('traffic_volume', axis=1)
y = encoded_data['traffic_volume']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize Numerical Features
# Now apply StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
data['holiday'] = data['holiday'].fillna('None')

In [ ]:
data['date_time'] = pd.to_datetime(data['date_time'])
data['hour'] = data['date_time'].dt.hour
data['day_of_week'] = data['date_time'].dt.dayofweek  # 0: Monday, 6: Sunday
data['month'] = data['date_time'].dt.month

In [ ]:
data = pd.get_dummies(data, columns=['weather_main', 'weather_description', 'holiday'], drop_first=True)

In [ ]:
data.head()

In [ ]:
data.drop(['date_time'], axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Separate features (X) and target (y)
X = data.drop('traffic_volume', axis=1)
y = data['traffic_volume']

# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler to the features and transform them
X_scaled = scaler.fit_transform(X)

# Convert the scaled features back to a DataFrame
X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

Model 1 Random Forest Model


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Create and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
print('Random Forest MAE:', mean_absolute_error(y_test, y_pred))
print('Random Forest RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('Random Forest R2:', r2_score(y_test, y_pred))

Evaluate Random Forest Model

XGBoost Model

In [ ]:
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

In [ ]:
print('XGBoost MAE:', mean_absolute_error(y_test, y_pred_xgb))
print('XGBoost RMSE:', np.sqrt(mean_squared_error(y_test, y_pred_xgb)))
print('XGBoost R2:', r2_score(y_test, y_pred_xgb))

Hyperparameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=XGBRegressor(random_state=42),
                           param_grid=param_grid,
                           scoring='neg_mean_squared_error',
                           cv=5,
                           verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters:", best_params)

# Train the model with the best parameters
best_xgb_model = XGBRegressor(**best_params, random_state=42)
best_xgb_model.fit(X_train, y_train)

# Evaluate the model
y_pred_xgb = best_xgb_model.predict(X_test)
print('XGBoost MAE:', mean_absolute_error(y_test, y_pred_xgb))
print('XGBoost RMSE:', np.sqrt(mean_squared_error(y_test, y_pred_xgb)))
print('XGBoost R2:', r2_score(y_test, y_pred_xgb))

LSTM Model Preparation

In [ ]:
X_train_lstm = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

Build LSTM Model

In [ ]:
from tensorflow.keras.layers import Input
# Build LSTM Model
lstm_model = Sequential()
lstm_model.add(Input(shape=(1, X_train.shape[1])))
lstm_model.add(LSTM(50, activation='relu'))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')


Train LSTM Model

In [ ]:
history = lstm_model.fit(X_train_lstm, y_train, epochs=50, validation_split=0.2, verbose=1)

Plot LSTM Training Loss

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('LSTM Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Evaluate LSTM Model

In [ ]:
y_pred_lstm = lstm_model.predict(X_test_lstm)
print('LSTM MAE:', mean_absolute_error(y_test, y_pred_lstm))
print('LSTM RMSE:', np.sqrt(mean_squared_error(y_test, y_pred_lstm)))
print('LSTM R2:', r2_score(y_test, y_pred_lstm))

Compare Model Performance

In [ ]:
models = ['Random Forest', 'XGBoost', 'LSTM']
mae_scores = [mean_absolute_error(y_test, y_pred), mean_absolute_error(y_test, y_pred_xgb), mean_absolute_error(y_test, y_pred_lstm)]
plt.figure(figsize=(10, 6))
sns.barplot(x=models, y=mae_scores)
plt.title('Model Comparison - MAE')
plt.xlabel('Model')
plt.ylabel('MAE')
plt.show()

Feature Importance - Random Forest

In [ ]:
importances = rf_model.feature_importances_
feature_names = X.columns
plt.figure(figsize=(12, 8))
sns.barplot(x=importances, y=feature_names)
plt.title('Feature Importance - Random Forest')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()